In [208]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from pylab import rcParams
%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LogisticRegression, LinearRegression, Ridge, Lasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBRegressor, XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from hyperopt.pyll.stochastic import sample
import time

In [209]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [210]:
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [197]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [198]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [199]:
train.shape

(891, 12)

In [200]:
for i in train.columns:
    print(train[i].value_counts())

891    1
293    1
304    1
303    1
302    1
301    1
300    1
299    1
298    1
297    1
296    1
295    1
294    1
292    1
306    1
291    1
290    1
289    1
288    1
287    1
286    1
285    1
284    1
283    1
282    1
281    1
305    1
307    1
279    1
321    1
      ..
561    1
560    1
584    1
585    1
586    1
587    1
610    1
609    1
608    1
607    1
606    1
605    1
604    1
603    1
602    1
601    1
600    1
599    1
598    1
597    1
596    1
595    1
594    1
593    1
592    1
591    1
590    1
589    1
588    1
1      1
Name: PassengerId, Length: 891, dtype: int64
0    549
1    342
Name: Survived, dtype: int64
3    491
1    216
2    184
Name: Pclass, dtype: int64
Holm, Mr. John Fredrik Alexander                       1
Gustafsson, Mr. Johan Birger                           1
Carter, Miss. Lucile Polk                              1
Richards, Mrs. Sidney (Emily Hocking)                  1
Jacobsohn, Mrs. Sidney Samuel (Amy Frances Christy)    1
Newell, Miss. Marjor

Name: Cabin, Length: 147, dtype: int64
S    644
C    168
Q     77
Name: Embarked, dtype: int64


In [201]:
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [211]:
#pop cabin and name
train.pop("Cabin")
train.pop("Name")
test.pop("Cabin")
test.pop("Name")

#convert variables of object type to categorical

categorical_vars = {"Sex","Ticket","Embarked"}
for var in categorical_vars:  
    train[var] = train[var].astype("category")
    test[var] = test[var].astype("category")

train.dtypes

PassengerId       int64
Survived          int64
Pclass            int64
Sex            category
Age             float64
SibSp             int64
Parch             int64
Ticket         category
Fare            float64
Embarked       category
dtype: object

In [212]:
X_train_kaggle = train.copy()
X_train_kaggle.pop("Survived")
y_train_kaggle = train["Survived"]
print(X_train_kaggle.shape, y_train_kaggle.shape)

X_train, X_val, y_train, y_val = train_test_split(X_train_kaggle, y_train_kaggle, test_size=0.3)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

X_test = test.copy()
print(X_test.shape)

(891, 9) (891,)
(623, 9) (623,)
(268, 9) (268,)
(418, 9)


In [204]:
#Manual feature preprocessing
X_train2 = train.copy()
SE1= SimpleImputer(strategy='median')
X_train2["Age"] = SE1.fit_transform(X_train2[["Age"]]).ravel()

SE2= SimpleImputer(strategy='most_frequent')
X_train2["Embarked"] = SE2.fit_transform(X_train2[["Embarked"]]).ravel()

X_train2["Embarked"] = X_train2["Embarked"].astype("category")

TE1 = TargetEncoder()
TE2 = TargetEncoder()
TE3 = TargetEncoder()

X_train2["Sex"] = TE1.fit_transform(X_train2["Sex"], X_train2["Survived"])
X_train2["Ticket"] = TE2.fit_transform(X_train2["Ticket"], X_train2["Survived"])
X_train2["Embarked"] = TE3.fit_transform(X_train2["Embarked"], X_train2["Survived"])

In [213]:
#Feature processing using Pipeline and fitting multiple models
X_train.pop("Ticket")
X_val.pop("Ticket")

numeric_transformer = Pipeline(steps=[           
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())])
categorical_features = X_train.select_dtypes(include=['category']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    XGBClassifier(random_state=1,learning_rate=0.05),
    lgb.LGBMClassifier(learning_rate = 0.15, metric = 'l1', n_estimators = 24),
    CatBoostClassifier()
    ]

for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    #pipe.fit(X_train, y_train)   
    print(classifier)
    print(cross_val_score(pipe, X_train, y_train, cv=5).mean())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')
0.7865161290322581
SVC(C=0.025, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
0.6227870967741935
NuSVC(cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
      kernel='rbf', max_iter=-1, nu=0.5, probability=True, random_state=None,
      shrinking=True, tol=0.001, verbose=False)
0.828258064516129
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_sam

84:	learn: 0.4230340	total: 2.48s	remaining: 26.7s
85:	learn: 0.4220306	total: 2.51s	remaining: 26.7s
86:	learn: 0.4210349	total: 2.54s	remaining: 26.7s
87:	learn: 0.4205871	total: 2.57s	remaining: 26.6s
88:	learn: 0.4194415	total: 2.6s	remaining: 26.6s
89:	learn: 0.4185986	total: 2.63s	remaining: 26.6s
90:	learn: 0.4181469	total: 2.66s	remaining: 26.6s
91:	learn: 0.4173603	total: 2.68s	remaining: 26.5s
92:	learn: 0.4154471	total: 2.71s	remaining: 26.5s
93:	learn: 0.4152733	total: 2.73s	remaining: 26.3s
94:	learn: 0.4149021	total: 2.76s	remaining: 26.3s
95:	learn: 0.4136148	total: 2.79s	remaining: 26.3s
96:	learn: 0.4131869	total: 2.82s	remaining: 26.2s
97:	learn: 0.4123958	total: 2.84s	remaining: 26.1s
98:	learn: 0.4116365	total: 2.86s	remaining: 26.1s
99:	learn: 0.4113587	total: 2.88s	remaining: 25.9s
100:	learn: 0.4110739	total: 2.89s	remaining: 25.7s
101:	learn: 0.4098347	total: 2.92s	remaining: 25.7s
102:	learn: 0.4088487	total: 2.95s	remaining: 25.7s
103:	learn: 0.4083601	total: 

243:	learn: 0.3515816	total: 6.87s	remaining: 21.3s
244:	learn: 0.3513428	total: 6.9s	remaining: 21.3s
245:	learn: 0.3511590	total: 6.92s	remaining: 21.2s
246:	learn: 0.3508444	total: 6.95s	remaining: 21.2s
247:	learn: 0.3506270	total: 6.98s	remaining: 21.2s
248:	learn: 0.3503712	total: 7.02s	remaining: 21.2s
249:	learn: 0.3500109	total: 7.04s	remaining: 21.1s
250:	learn: 0.3498281	total: 7.08s	remaining: 21.1s
251:	learn: 0.3496381	total: 7.1s	remaining: 21.1s
252:	learn: 0.3493385	total: 7.13s	remaining: 21s
253:	learn: 0.3491327	total: 7.16s	remaining: 21s
254:	learn: 0.3487254	total: 7.19s	remaining: 21s
255:	learn: 0.3481951	total: 7.22s	remaining: 21s
256:	learn: 0.3478152	total: 7.25s	remaining: 21s
257:	learn: 0.3477025	total: 7.28s	remaining: 20.9s
258:	learn: 0.3475272	total: 7.31s	remaining: 20.9s
259:	learn: 0.3469831	total: 7.34s	remaining: 20.9s
260:	learn: 0.3466471	total: 7.38s	remaining: 20.9s
261:	learn: 0.3462101	total: 7.41s	remaining: 20.9s
262:	learn: 0.3459442	to

402:	learn: 0.3196470	total: 11.4s	remaining: 16.9s
403:	learn: 0.3195764	total: 11.5s	remaining: 16.9s
404:	learn: 0.3194629	total: 11.5s	remaining: 16.9s
405:	learn: 0.3191741	total: 11.5s	remaining: 16.9s
406:	learn: 0.3189637	total: 11.6s	remaining: 16.9s
407:	learn: 0.3187882	total: 11.6s	remaining: 16.8s
408:	learn: 0.3184921	total: 11.6s	remaining: 16.8s
409:	learn: 0.3184793	total: 11.6s	remaining: 16.8s
410:	learn: 0.3182035	total: 11.7s	remaining: 16.7s
411:	learn: 0.3180381	total: 11.7s	remaining: 16.7s
412:	learn: 0.3178336	total: 11.7s	remaining: 16.7s
413:	learn: 0.3177171	total: 11.8s	remaining: 16.6s
414:	learn: 0.3173603	total: 11.8s	remaining: 16.6s
415:	learn: 0.3172906	total: 11.8s	remaining: 16.6s
416:	learn: 0.3169650	total: 11.8s	remaining: 16.6s
417:	learn: 0.3167365	total: 11.9s	remaining: 16.5s
418:	learn: 0.3166221	total: 11.9s	remaining: 16.5s
419:	learn: 0.3163287	total: 11.9s	remaining: 16.5s
420:	learn: 0.3160155	total: 12s	remaining: 16.5s
421:	learn: 0.

561:	learn: 0.2940885	total: 16.2s	remaining: 12.6s
562:	learn: 0.2939832	total: 16.2s	remaining: 12.6s
563:	learn: 0.2938515	total: 16.3s	remaining: 12.6s
564:	learn: 0.2937116	total: 16.3s	remaining: 12.5s
565:	learn: 0.2936313	total: 16.3s	remaining: 12.5s
566:	learn: 0.2935586	total: 16.4s	remaining: 12.5s
567:	learn: 0.2933432	total: 16.4s	remaining: 12.5s
568:	learn: 0.2932345	total: 16.4s	remaining: 12.4s
569:	learn: 0.2929908	total: 16.4s	remaining: 12.4s
570:	learn: 0.2927701	total: 16.5s	remaining: 12.4s
571:	learn: 0.2926395	total: 16.5s	remaining: 12.4s
572:	learn: 0.2925195	total: 16.5s	remaining: 12.3s
573:	learn: 0.2921796	total: 16.6s	remaining: 12.3s
574:	learn: 0.2919316	total: 16.6s	remaining: 12.3s
575:	learn: 0.2917451	total: 16.6s	remaining: 12.2s
576:	learn: 0.2916505	total: 16.7s	remaining: 12.2s
577:	learn: 0.2913896	total: 16.7s	remaining: 12.2s
578:	learn: 0.2913202	total: 16.7s	remaining: 12.2s
579:	learn: 0.2912017	total: 16.8s	remaining: 12.1s
580:	learn: 

...

223:	learn: 0.3468988	total: 6.51s	remaining: 22.6s
224:	learn: 0.3467390	total: 6.55s	remaining: 22.6s
225:	learn: 0.3465556	total: 6.57s	remaining: 22.5s
226:	learn: 0.3462931	total: 6.6s	remaining: 22.5s
227:	learn: 0.3458870	total: 6.63s	remaining: 22.5s
228:	learn: 0.3456570	total: 6.67s	remaining: 22.4s
229:	learn: 0.3452778	total: 6.7s	remaining: 22.4s
230:	learn: 0.3450630	total: 6.73s	remaining: 22.4s
231:	learn: 0.3449727	total: 6.75s	remaining: 22.3s
232:	learn: 0.3446887	total: 6.78s	remaining: 22.3s
233:	learn: 0.3443198	total: 6.81s	remaining: 22.3s
234:	learn: 0.3442092	total: 6.84s	remaining: 22.3s
235:	learn: 0.3439379	total: 6.88s	remaining: 22.3s
236:	learn: 0.3436958	total: 6.91s	remaining: 22.2s
237:	learn: 0.3434723	total: 6.94s	remaining: 22.2s
238:	learn: 0.3433387	total: 6.96s	remaining: 22.2s
239:	learn: 0.3431075	total: 6.99s	remaining: 22.1s
240:	learn: 0.3427708	total: 7.03s	remaining: 22.1s
241:	learn: 0.3423678	total: 7.06s	remaining: 22.1s
242:	learn: 0.

382:	learn: 0.3099748	total: 11.5s	remaining: 18.5s
383:	learn: 0.3095184	total: 11.5s	remaining: 18.4s
384:	learn: 0.3094791	total: 11.5s	remaining: 18.4s
385:	learn: 0.3091215	total: 11.6s	remaining: 18.4s
386:	learn: 0.3090383	total: 11.6s	remaining: 18.3s
387:	learn: 0.3090165	total: 11.6s	remaining: 18.3s
388:	learn: 0.3087400	total: 11.6s	remaining: 18.3s
389:	learn: 0.3085129	total: 11.7s	remaining: 18.2s
390:	learn: 0.3084225	total: 11.7s	remaining: 18.2s
391:	learn: 0.3082123	total: 11.7s	remaining: 18.2s
392:	learn: 0.3081240	total: 11.7s	remaining: 18.1s
393:	learn: 0.3080093	total: 11.8s	remaining: 18.1s
394:	learn: 0.3078185	total: 11.8s	remaining: 18.1s
395:	learn: 0.3076420	total: 11.8s	remaining: 18.1s
396:	learn: 0.3073936	total: 11.9s	remaining: 18s
397:	learn: 0.3073574	total: 11.9s	remaining: 18s
398:	learn: 0.3069530	total: 11.9s	remaining: 18s
399:	learn: 0.3065860	total: 12s	remaining: 17.9s
400:	learn: 0.3062454	total: 12s	remaining: 17.9s
401:	learn: 0.3061650	

541:	learn: 0.2832422	total: 16.2s	remaining: 13.7s
542:	learn: 0.2829622	total: 16.2s	remaining: 13.7s
543:	learn: 0.2828464	total: 16.3s	remaining: 13.6s
544:	learn: 0.2827060	total: 16.3s	remaining: 13.6s
545:	learn: 0.2825537	total: 16.3s	remaining: 13.6s
546:	learn: 0.2824277	total: 16.3s	remaining: 13.5s
547:	learn: 0.2821723	total: 16.4s	remaining: 13.5s
548:	learn: 0.2821454	total: 16.4s	remaining: 13.5s
549:	learn: 0.2820648	total: 16.4s	remaining: 13.4s
550:	learn: 0.2818356	total: 16.5s	remaining: 13.4s
551:	learn: 0.2814701	total: 16.5s	remaining: 13.4s
552:	learn: 0.2813852	total: 16.5s	remaining: 13.4s
553:	learn: 0.2811356	total: 16.6s	remaining: 13.3s
554:	learn: 0.2811140	total: 16.6s	remaining: 13.3s
555:	learn: 0.2807322	total: 16.6s	remaining: 13.3s
556:	learn: 0.2806803	total: 16.6s	remaining: 13.2s
557:	learn: 0.2804814	total: 16.7s	remaining: 13.2s
558:	learn: 0.2802527	total: 16.7s	remaining: 13.2s
559:	learn: 0.2800955	total: 16.7s	remaining: 13.2s
560:	learn: 

700:	learn: 0.2591292	total: 21.1s	remaining: 9.02s
701:	learn: 0.2590521	total: 21.2s	remaining: 8.99s
702:	learn: 0.2587621	total: 21.2s	remaining: 8.96s
703:	learn: 0.2587055	total: 21.2s	remaining: 8.93s
704:	learn: 0.2586869	total: 21.3s	remaining: 8.9s
705:	learn: 0.2584213	total: 21.3s	remaining: 8.87s
706:	learn: 0.2583399	total: 21.3s	remaining: 8.84s
707:	learn: 0.2580884	total: 21.4s	remaining: 8.81s
708:	learn: 0.2577544	total: 21.4s	remaining: 8.78s
709:	learn: 0.2574282	total: 21.4s	remaining: 8.75s
710:	learn: 0.2573301	total: 21.5s	remaining: 8.72s
711:	learn: 0.2571665	total: 21.5s	remaining: 8.7s
712:	learn: 0.2570064	total: 21.5s	remaining: 8.67s
713:	learn: 0.2567492	total: 21.6s	remaining: 8.64s
714:	learn: 0.2564883	total: 21.6s	remaining: 8.61s
715:	learn: 0.2564587	total: 21.6s	remaining: 8.58s
716:	learn: 0.2563340	total: 21.7s	remaining: 8.55s
717:	learn: 0.2561667	total: 21.7s	remaining: 8.52s
718:	learn: 0.2560694	total: 21.7s	remaining: 8.49s
719:	learn: 0.

859:	learn: 0.2377275	total: 26.2s	remaining: 4.27s
860:	learn: 0.2376675	total: 26.2s	remaining: 4.24s
861:	learn: 0.2375499	total: 26.3s	remaining: 4.21s
862:	learn: 0.2374487	total: 26.3s	remaining: 4.18s
863:	learn: 0.2373127	total: 26.3s	remaining: 4.14s
864:	learn: 0.2372500	total: 26.4s	remaining: 4.12s
865:	learn: 0.2371693	total: 26.4s	remaining: 4.08s
866:	learn: 0.2370970	total: 26.4s	remaining: 4.05s
867:	learn: 0.2370056	total: 26.5s	remaining: 4.02s
868:	learn: 0.2369030	total: 26.5s	remaining: 3.99s
869:	learn: 0.2365348	total: 26.5s	remaining: 3.96s
870:	learn: 0.2364464	total: 26.6s	remaining: 3.93s
871:	learn: 0.2364335	total: 26.6s	remaining: 3.9s
872:	learn: 0.2364189	total: 26.6s	remaining: 3.87s
873:	learn: 0.2362933	total: 26.7s	remaining: 3.84s
874:	learn: 0.2362114	total: 26.7s	remaining: 3.81s
875:	learn: 0.2361186	total: 26.7s	remaining: 3.78s
876:	learn: 0.2359293	total: 26.8s	remaining: 3.75s
877:	learn: 0.2359058	total: 26.8s	remaining: 3.72s
878:	learn: 0

In [214]:
#Hyperparameter tuning for most optimal model
finalpipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', lgb.LGBMClassifier())])   

params = {  
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(1)),
    'max_depth': scope.int(hp.quniform('max_depth', 5, 15, 1)),
    'n_estimators': scope.int(hp.quniform('n_estimators', 5, 35, 1)),
    'num_leaves': scope.int(hp.quniform('num_leaves', 5, 50, 1)),
    'boosting_type': hp.choice('boosting_type', ['gbdt', 'dart']),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0)}

In [235]:
def hyperopt(param_space, X_train, y_train, X_test, y_test, num_eval):
    
    start = time.time()
    
    def objective_function(params):
        clf = lgb.LGBMClassifier(**params)
        score = cross_val_score(clf, X_train, y_train, cv=5).mean()
        return {'loss': -score, 'status': STATUS_OK}

    trials = Trials()
    best_param = fmin(objective_function, 
                      param_space, 
                      algo=tpe.suggest, 
                      max_evals=num_eval, 
                      trials=trials,
                      rstate= np.random.RandomState(1))
    loss = [x['result']['loss'] for x in trials.trials]
    
    best_param_values = [x for x in best_param.values()]
    
    if best_param_values[0] == 0:
        boosting_type = 'gbdt'
    else:
        boosting_type= 'dart'
    
    clf_best = lgb.LGBMClassifier(learning_rate=best_param_values[2],
                                  num_leaves=int(best_param_values[5]),
                                  max_depth=int(best_param_values[3]),
                                  n_estimators=int(best_param_values[4]),
                                  boosting_type=boosting_type,
                                  colsample_bytree=best_param_values[1],
                                  reg_lambda=best_param_values[6],
                                 )
    clf_best.fit(X_train, y_train)
    joblib.dump(clf_best, 'finalmodel2.pkl')
    
    print("")
    print("##### Results")
    print("Score best parameters: ", min(loss)*-1)
    print("Best parameters: ", best_param)
    print("Test Score: ", clf_best.score(X_test, y_test))
    print("Time elapsed: ", time.time() - start)
    print("Parameter combinations evaluated: ", num_eval)
    
    return trials, clf_best

In [236]:
num_eval =75
results_hyperopt = hyperopt(params, X_train, y_train, X_val, y_val, num_eval)

100%|██████████████████████████████████████████████████| 75/75 [00:20<00:00,  3.47it/s, best loss: -0.8330193548387097]

##### Results
Score best parameters:  0.8330193548387097
Best parameters:  {'boosting_type': 1, 'colsample_by_tree': 0.6736658466347957, 'learning_rate': 0.2384421758969943, 'max_depth': 15.0, 'n_estimators': 21.0, 'num_leaves': 50.0, 'reg_lambda': 0.9174644025214022}
Test Score:  0.7985074626865671
Time elapsed:  20.68965458869934
Parameter combinations evaluated:  75


In [249]:
# load model from disk for predictions
loaded_model = joblib.load('finalmodel.pkl')
predictions = loaded_model.predict(X_test)

ValueError: could not convert string to float: 'male'

Prediction on X_test does not work because X_test has not been preprocessed. Can apply pipeline used to fit the training set on test set as well.

In [238]:
predictpipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', XGBClassifier())]) 

predictpipe.fit(X_train, y_train)
predictions = predictpipe.predict(X_test)

In [245]:
#Create a DataFrame with the passenger ids and our prediction regarding whether they will survive or not
submission = pd.DataFrame({'PassengerID':X_test["PassengerId"],'Survived':predictions})

#Visualize the first 5 rows
submission.head()

,PassengerID,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [247]:
filename = 'Kaggle Prob 2 Predictions.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: Kaggle Prob 2 Predictions.csv
